In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [7]:
X=scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [11]:
model=Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 4s 48ms/step - loss: 0.6583 - accuracy: 0.6319 - val_loss: 0.6177 - val_accuracy: 0.7013
Epoch 2/100
20/20 [==============================] - 0s 16ms/step - loss: 0.5995 - accuracy: 0.6824 - val_loss: 0.5727 - val_accuracy: 0.7662
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5619 - accuracy: 0.7182 - val_loss: 0.5445 - val_accuracy: 0.7532
Epoch 4/100
20/20 [==============================] - 0s 11ms/step - loss: 0.5391 - accuracy: 0.7345 - val_loss: 0.5273 - val_accuracy: 0.7727
Epoch 5/100
20/20 [==============================] - 0s 20ms/step - loss: 0.5231 - accuracy: 0.7329 - val_loss: 0.5130 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 19ms/step - loss: 0.5110 - accuracy: 0.7378 - val_loss: 0.5030 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 15ms/step - loss: 0.5017 - accuracy: 0.7410 - val_loss: 0.4949 - val_accuracy: 0.7857
Epoch 8

In [14]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.8 MB/s eta 0:00:00


In [15]:
import keras_tuner as kt

In [16]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [17]:
tuner=kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

In [18]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 10s


In [19]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 33ms/step - loss: 0.4193 - accuracy: 0.7948 - val_loss: 0.4562 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 14ms/step - loss: 0.4189 - accuracy: 0.7964 - val_loss: 0.4547 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 14ms/step - loss: 0.4190 - accuracy: 0.7964 - val_loss: 0.4541 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4184 - accuracy: 0.7899 - val_loss: 0.4530 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4186 - accuracy: 0.7932 - val_loss: 0.4536 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4180 - accuracy: 0.7948 - val_loss: 0.4537 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4174 - accuracy: 0.7980 - val_loss: 0.4541 - val_accuracy: 0.8117
Epoch 

In [24]:
def build_model1(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [28]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='prj1')

In [29]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 10s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'units': 56}

In [31]:
model=tuner.get_best_models(num_models=1)[0]